In [1]:
import csv
import openpyxl
import os
class subject:
    
    def __init__(self,subno,subname,ltp,crd):
        self.subno=subno
        self.subname=subname
        self.ltp=ltp
        self.crd=crd

class sem:
    
    def __init__(self):
        self.spi=0.0
        self.courses=[]
        self.credits=0.0

class student:
    
    def __init__(self):
        self.roll=''
        self.name=''
        self.department=''
        self.cpi=[0 for i in range(8)]
        self.sem_data=list(None for i in range(8))
        self.total_credits=[0 for i in range(8)]
        
class course:
    
    def __init__(self,subj,grade,tp):
        self.subj=subj
        self.grade=grade
        self.tp=tp

file="sample_input/names-roll.csv"
rn=dict()
with open(file, 'r') as csvfile:
    csvreader = (csv.reader(csvfile))
    next(csvreader)
    for x in (csvreader):
        k=student()
        k.roll=x[0]
        k.name=x[1]
        rn[x[0]]=k
        k.department=x[0][4:6]
    
file="sample_input/subjects_master.csv"
sub=dict()
with open(file, 'r') as csvfile:
    csvreader = (csv.reader(csvfile))
    next(csvreader)
    for x in csvreader:
        k=subject(x[0],x[1],x[2],int(x[3]))
        sub[x[0]]=k
    
file="sample_input/grades.csv"
with open(file, 'r') as csvfile:
    csvreader = (csv.reader(csvfile))
    next(csvreader)
    for x in csvreader:
        if not rn[x[0]].sem_data[int(x[1])-1]:
            rn[x[0]].sem_data[int(x[1])-1]=sem()
        sd=rn[x[0]].sem_data[int(x[1])-1]
        sd.credits=sd.credits+int(x[3])
        sd_courses=sd.courses
        k=course(sub[x[2]].subno,x[4],x[5])
        sd_courses.append(k)
        #print(sd_courses)

for x in rn:
    file=x+'.xlsx'
    if os.path.exists(file):
        os.remove(file)
    
def caltotcredits(s):
    sd=s.sem_data
    s.total_credits[0]=sd[0].credits
    for i in range(1,8,1):
        if sd[i]!=None:
            s.total_credits[i]=s.total_credits[i-1]+sd[i].credits
        else:
            s.total_credits[i]=s.total_credits[i-1]
            
def calspicpi(s):
    sd=s.sem_data
    data={'AA':10,'AB':9,'BB':8,'BC':7,'CC':6,'CD':5,'DD':4,'F':0,'F*':0,'I':0}
    for i in range(8):
        if sd[i]!=None:
            courses=sd[i].courses
            val=0
            for subs in courses:
                try:
                    sg=subs.grade
                    sg=sg.replace(" ", "")
                    val=data[sg[0:2]]*sub[subs.subj].crd +val
                except:
                    print(s.roll,i,subs.subj,subs.grade,len(subs.grade))
            sd[i].spi=val
            if i==0:
                s.cpi[0]=val
            else:
                s.cpi[i]=s.cpi[i-1]+sd[i].spi  
            #print(subs.subj,subs.grade,val,sd[i].credits)
                
#calspicpi(rn['0401CS02'])

c = "./output/"
try:
    os.mkdir(c)
except:
    pass

for x in rn:
    caltotcredits(rn[x])
    calspicpi(rn[x])
    wb=openpyxl.Workbook()
    b=wb.active
    b.title="Overall"
    l=['Roll No','Name of Student','Discipline','Semester No','Semester wise Credit Taken','SPI','Total Credits Taken','CPI']
    for i in range(1,len(l)+1):
        v='A'+str(i)
        c=b[v]
        c.value=l[i-1]
    c=b['B1']
    c.value=x
    c=b['B2']
    c.value=rn[x].name
    c=b['B3']
    c.value=rn[x].department
    d=['B','C','D','E','F','G','H','I']
    sd=rn[x].sem_data
    for i in range(8):
        v=d[i]+'4'
        c=b[v]
        c.value=i+1
        v=d[i]+'5'
        c=b[v]
        if sd[i]!=None:
            c.value=sd[i].credits
        v=d[i]+'6'
        c=b[v]
        if sd[i]!=None:
            c.value=round((sd[i].spi)/sd[i].credits,2)
        v=d[i]+'7'
        c=b[v]
        c.value=rn[x].total_credits[i]
        v=d[i]+'8'
        c=b[v]
        c.value=round((rn[x].cpi[i])/rn[x].total_credits[i],2)
        
    for i in range(8):
        t='Sem'+str(i+1)
        wb.create_sheet(index = i+1 , title = t)
        bb=wb[t]  #wb.get_sheet_by_name(t)
        l=['Subject No.','Subject Name','L-T-P','Credit','Grade']
        d1=['A','B','C','D','E']
        for j in range(len(l)):
            v=d1[j]+'1'
            c=bb[v]
            c.value=l[j]
        sd=rn[x].sem_data[i]
        courses=[]
        if sd!=None:
            courses=sd.courses
        
        for k in range(len(courses)):
            sub_d=sub[courses[k].subj]
            CC={'A':sub_d.subno,'B':sub_d.subname,'C':sub_d.ltp,'D':sub_d.crd,'E':courses[k].grade}
            for j in range(len(d1)):
                v=d1[j]+str(k+2)
                c=bb[v]
                c.value=CC[d1[j]]
                
    wb.save('./output/' + x +'.xlsx')


In [2]:
import pandas as pd
from fpdf import FPDF
import csv
import xlsxwriter
from openpyxl import*
from openpyxl.styles import*
import os
import PySimpleGUI as sg
from win32com import client

In [3]:
def set_border(ws,rs,cs,arr):
    
    border = Border(left=Side(border_style='thin', color='000000'),
                right=Side(border_style='thin', color='000000'),
                top=Side(border_style='thin', color='000000'),
                bottom=Side(border_style='thin', color='000000'))

    for row in range(1,arr.shape[0]+2):
        for col in range(arr.shape[1]):
            cell=ws[row+rs][col+cs]
            cell.border = border

In [4]:
def findcolswidth(ws,r,c):
    m=0
    for i in range(10,r,1):
        cell=str(ws[i][c].value)
        m=max(m,len(cell))
    return m

In [5]:
def styledf(ws,rs,cs,arr):
    
    align= Alignment(horizontal="center",vertical="center")
    font=Font(size=7)
    fontb=Font(size=7,bold=True)

    
    for i in range(1,arr.shape[0]+2):
        for j in range(arr.shape[1]):
            cell=ws[i+rs][j+cs]
            #print(cell.value)
            cell.alignment=align
            cell.font=font
            if i==1:
                cell.font=fontb
        

In [6]:
def style2(dfl,file):
    
    wb=load_workbook(file)
    ws=wb.active
    
    #ws.merge_cells('A2:D4')
    
    row=10
    col=0
    
    
    for i in range(1,len(dfl),1):
        
        if i%2==1:
            
            styledf(ws,row,col,dfl[i])
            set_border(ws,row,col,dfl[i])
            col=dfl[i].shape[1]
            
        else:
            
            styledf(ws,row,col+2,dfl[i])
            set_border(ws,row,col+2,dfl[i])
            row=row+max(dfl[i].shape[0],dfl[i-1].shape[0])+2
            col=0
            
    wb.save(file)
    wb.close()

In [7]:
def insert_image(output_file):
    
    wb = load_workbook(output_file)
    ws = wb.active
    img =openpyxl.drawing.image.Image('logo.jpeg')
    img.width=800
    img.height=120
    ws.add_image(img)
    
    align= Alignment(horizontal="center",vertical="center")
    fontb=Font(size=7,bold=True)
    
    ws.merge_cells('C8:E8')
    ws.merge_cells('C9:E9')
    ws.merge_cells('F8:I8')
    ws.merge_cells('F9:I9')
    
    
    l=["Name :","Roll No :", "Course :"]
    for i in range(len(l)):
        c='A'+str(i+8)
        ws[c].value=l[i]
        ws[c].font=fontb
        ws[c].alignment=align
    y=["Year of Admission: ","Programme :"]
    for i in range(len(y)):
        c='C'+str(i+8)
        ws[c].value=y[i]
        ws[c].font=fontb
        ws[c].alignment=align
        
    ws['F9']=""
    ws['F9'].font=Font(size=7)
    
    
    
    wb.save(output_file)
    wb.close()
    

In [8]:
def insert_img(pimage,output_file,pos,wid,hei):
    wb = load_workbook(output_file)
    ws = wb.active
    img =openpyxl.drawing.image.Image(pimage)
    img.width=wid
    img.height=hei
    img.anchor = pos
    ws.add_image(img)
    wb.save(output_file)
    wb.close()
    

In [9]:
def mergetosingle(dfl,file_name,psign,pstamp):
    
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')
    
    row=10
    col=0
    
    for i in range(1,len(dfl),1):
        if i%2==1:
            dfl[i].to_excel(writer,sheet_name="Sheet1", startrow=row, startcol=col,index=False)
            col=dfl[i].shape[1]
        else:
            dfl[i].to_excel(writer,sheet_name="Sheet1", startrow=row, startcol=col+2,index=False)
            row=row+max(dfl[i].shape[0],dfl[i-1].shape[0])+2
            col=0
            
    writer.save()  
    
    insert_image(file_name)
    insert_img(pstamp,file_name,'C60',200,200)
    insert_img(psign,file_name,'K60',120,120)

In [10]:
def read_input(values):
    start=int(values['from'][-2:])
    end=int(values['to'][-2:])
    path=values['sign']
    path=path[0:-9]
    st=values['from']
    
    print(path)
    
    for i in range(start,end+1,1):
        k=str(i)
        if len(k)!=2:
            k='0'+k  
        k=st[0:-2]+k
        makeexcel(k,path,values['sign'],values['stamp'])
    

In [11]:
def  makeexcel(r,path,psign,pstamp): 
    
    file=path+'output/'+r+'.xlsx'

    excel_file = pd.ExcelFile(file)
    sheets = excel_file.sheet_names

    df_total={}
    i=0
    for sheet in sheets:               # loop through sheets inside an Excel file
        df= pd.read_excel(excel_file, sheet_name=sheet)
        df_total[i]=df
        i=i+1

    #print(df_total)  

    output_file=r+'.xlsx'

    if os.path.exists(output_file):
        os.remove(output_file)  

    mergetosingle(df_total,output_file,psign,pstamp)
    style2(df_total,output_file)

    wb=load_workbook(output_file)
    ws=wb.active

    for i in range(11):
        c=chr(ord('A')+i)
        ws.column_dimensions[c].width =findcolswidth(ws,50,i)*0.7
    
    wb.save(output_file)
    file=path+output_file   
    makepdf(r,file)

In [12]:
# Import Module

def makepdf(r,path):
    # Open Microsoft Excel
    excel = client.Dispatch('Excel.Application')
    # Read Excel File
    wb= excel.Workbooks.Open(path)
    ws = wb.Worksheets[0]

    # Convert into PDF File
    ws.PageSetup.paperSize=8
    pdf_path=path[:-5]+'.pdf'
    wb.SaveAs(pdf_path, FileFormat=57)
    wb.Close(True)
    excel.Quit()


In [13]:
#makeexcel('0401CS01','','','')
# file='C:/Users/vasavi/Desktop/Baggu_Assignment/tut_2/updated.xlsx'
# makepdf('0401CS01',file)
# excel = client.Dispatch('Excel.Application')
# excel.quit()

In [ ]:
sg.theme('Dark2')

layout = [[sg.Text("Range of roll numbers "), 
           sg.Input(key='from'),sg.Input(key="to")],
          [sg.Text("Stamp:"), 
           sg.Input(key='stamp'),sg.FileBrowse()],
          [sg.Text("Sign:"), 
           sg.Input(key='sign'),sg.FileBrowse()],
          [sg.Button('generate')],
          [sg.Button('Exit')]
         ]

window = sg.Window('MarkSheet Generator', layout)

while True:
    
    event, values = window.read()

    #print(event, values)

    if event in (None, 'Exit'):
        break
    elif event=='generate':
        path=values['sign']
        print(path[0:-9])
        read_input(values)
#         try:
#             read_input(values)
#             break
#         except:
#             window.close()
            
        
window.close()

In [15]:
window.close()